# Machine learning for sorting neurons

In [1]:
# Load the libraries
import caiman as cm
from caiman.source_extraction import cnmf

import matplotlib.pyplot as plt
from matplotlib.widgets import Button, TextBox
from matplotlib.patches import Polygon
from matplotlib.path import Path
import matplotlib.gridspec as gridspec

import numpy as np
import pandas as pd
import json
from scipy.sparse import csc_matrix
from scipy.stats import sem
from scipy import stats
import statistics

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn import linear_model as lm

import cv2
from itertools import compress

from IPython import get_ipython
import IPython

from ipyfilechooser import FileChooser

from scipy.ndimage import binary_erosion
from scipy.ndimage import grey_erosion

try:
    if __IPYTHON__:
        get_ipython().run_line_magic('load_ext', 'autoreload')
        get_ipython().run_line_magic('autoreload', '2')
except NameError:
    pass

### Plan for the minimum amount of data for training:
Minimun for meaningful training:
* 3 recordings for train/test split
* 2 recordings for pure test

Needed files:
* component_locations
* traces
* img_filt
* manually selected

In [47]:
# Load the data
# Load the data
paths = ['D:/CaIm/os80/b2/']
         #'D:/CaIm/os82/b22/']

# for now, only traces
components_all = pd.DataFrame(columns =  ["recording_name","trace","good_neu"])

for path in paths:
    traces_path = path + 'traces.json'
    component_locations_path = path + 'component_locations.json'
    filter_img_path = path + 'filter_img.json'
    good_neu_path = path + 'good_neurons.csv' # CHANGE TO GOOD NEURONS
    with open(traces_path, 'r') as json_file:
        traces_list = json.load(json_file)

    good_neu = pd.read_csv(good_neu_path)
    good_neu = good_neu["is_good"].tolist()
    ''' 
    with open(component_locations_path, 'r') as json_file:
        component_locations_list = json.load(json_file)
    with open(filter_img_path, 'r') as json_file: 
        filter_img = json.load(json_file)
        component_locations = np.array(component_locations_list)
    component_locations_sparse = csc_matrix(component_locations)
        filter_img = np.array(filter_img)
    '''
    # Convert to convinient formats
    traces = np.array(traces_list)
    components_add = pd.DataFrame( columns =  [ "recording_name","trace","good_neu"])
    

    components_add["trace"] = traces_list
    components_add["good_neu"] =  good_neu
    components_add["recording_name"] = path[8:-1]
    # Display the number of components total and the rejected ones
    print(f"Total number of components: {traces.shape[0]} ")
    components_all = pd.concat([components_all, components_add], ignore_index = True) 
    components_all.reset_index() 
          
    # Select only accepted:
    #component_locations = component_locations[:,accepted_list]
components_all

Total number of components: 466 


,recording_name,trace,good_neu
0,os80/b2,"[-46.83468517863951, 3.932728674972347, -11.63...",0
1,os80/b2,"[-50.37886331376455, -59.49971205606676, -67.7...",1
2,os80/b2,"[37.760619901980874, 114.33621150657424, 287.5...",0
3,os80/b2,"[378.6210203997481, 326.37862097594694, 278.66...",0
4,os80/b2,"[483.6057513720241, 565.1819551079657, 594.187...",0
...,...,...,...
461,os80/b2,"[274.79127558521, 234.5597408718737, 208.60979...",0
462,os80/b2,"[-384.7995528951263, -392.99651318134823, -400...",0
463,os80/b2,"[211.72553651160447, 180.40662836004452, 151.6...",0
464,os80/b2,"[1461.986112104748, 1771.3844334855155, 1638.4...",1


## Extract all the relevant statistical features
* Basic stats: st dev, mean, median, skew, kurtosis
* Difference between st dev in different segments of the recording
* Some fancy measure of chaos
* Rise/fall ratios
* 

# Cells from older scripts:

In [ ]:
## Evaluate roundness

In [ ]:
import cv2

def calculate_circularity(neuron):
    # Find area (non-zero pixels)
    area = np.count_nonzero(neuron)
    
    # Find perimeter (using edge detection, e.g., Canny)
  #  edges = cv2.Canny(neuron, threshold1=100, threshold2=200)
    
    neuron_bool = neuron.astype(bool)
    # Find boundaries (edges)
    eroded = binary_erosion(neuron_bool)
    boundaries = neuron_bool & ~eroded
    perimeter = np.count_nonzero(boundaries)
    
    if perimeter == 0:
        return 0  # To avoid division by zero
    
    # Calculate circularity
    circularity = 4 * np.pi * area / (perimeter ** 2)
    return circularity

circs = []
circs_sharp = []
for n in np.arange(traces.shape[0]):
    neuron = component_locations[:,n].reshape(d2, d1, order='F')
    neuron_sharp = component_locs_sharp[n].reshape(d2, d1, order='F')
    circ = calculate_circularity(neuron)
    circ_sharp = calculate_circularity(neuron_sharp)
    circs.append(circ)
    circs_sharp.append(circ_sharp)
  #  print(f"Neuron#{n+1} circularity reg: {round(circ,3)}, sharp:{round(circ_sharp,3)}")
    

n_bins = 20

fig, axs = plt.subplots(1, 2, sharey=True, tight_layout=True)

# We can set the number of bins with the *bins* keyword argument.
axs[0].hist(circs, bins=n_bins)
axs[1].hist(circs_sharp, bins=n_bins)

In [ ]:
diffs_good = []
diffs_mid = []
diffs_bad = []
neus = [[4,6,9, 32,58,65],[2,3,14, 15,18,26]]
for neu in neus[0]:
    diffs= []
    for n in np.arange(traces.shape[1]-1):
        diff = traces[neu][n+1] - traces[neu][n]
        diffs.append(diff)
    diffs_good.append(diffs)

for neu in neus[1]:
    diffs= []
    for n in np.arange(traces.shape[1]-1):
        diff = traces[neu][n+1] - traces[neu][n]
        diffs.append(diff)
    diffs_bad.append(diffs)

n_bins =50

fig, axs = plt.subplots(2,len (neus[0]) , sharey=True, tight_layout=True)

# We can set the number of bins with the *bins* keyword argument.

for n in np.arange( len (neus[0]) ):
    axs[0,n].hist(diffs_good[n], bins=n_bins)
    axs[1,n].hist(diffs_bad[n], bins=n_bins)

In [ ]:
snrs = []
st_devs = []
kurts = []
means = []
medians = []
skews = []
for n in np.arange(traces.shape[0]):
    st_dev = np.std(traces[n])
    mean_tr = np.mean(traces[n])
    snr = mean_tr/st_dev
    medians.append(np.median(traces[n]))
    means.append(mean_tr)
    kurts.append(stats.kurtosis(traces[n]))
    snrs.append(snr)
    st_devs.append(st_dev)
    skews.append(stats.skew(traces[n]))

In [ ]:
status_array_bool = status_array.astype(bool)
baddies = [not x for x in status_array_bool]
good_circs = list(compress(circs, status_array_bool))
bad_circs = list(compress(circs, baddies))
good_circs_sharp = list(compress(circs_sharp, status_array_bool))
bad_circs_sharp = list(compress(circs_sharp, baddies))
skews_good = list(compress(skews, status_array_bool))
skews_bad = list(compress(skews, baddies))

rise_fall_ratios

rfrs_good = list(compress(rise_fall_ratios, status_array_bool))
rfrs_bad = list(compress(rise_fall_ratios, baddies))

mean_spikes
mean_spikes_good = list(compress(mean_spikes, status_array_bool))
mean_spikes_bad = list(compress(mean_spikes, baddies))

n_bins = 20


fig, axs = plt.subplots(1, 2, sharey=True, tight_layout=True)

# We can set the number of bins with the *bins* keyword argument.

#bins = np.linspace(min(skews), max(skews), n_bins)
#axs[0].hist(bad_circs, alpha=0.5, bins=bins)

bins = np.linspace(min(mean_spikes), max(mean_spikes), n_bins)
axs[1].hist(mean_spikes_good, alpha=0.5, bins=bins)
axs[1].hist(mean_spikes_bad, alpha=0.5, bins=bins)
plt.show()

In [ ]:
snrs_good = list(compress(snrs, status_array_bool))
snrs_bad = list(compress(snrs, baddies))
st_devs_good = list(compress(st_devs, status_array_bool))
st_devs_bad = list(compress(st_devs, baddies))

kurts_good = list(compress(kurts, status_array_bool))
kurts_bad = list(compress(kurts, baddies))

means_good = list(compress(means, status_array_bool))
means_bad = list(compress(means, baddies))

medians_good = list(compress(medians, status_array_bool))
medians_bad = list(compress(medians, baddies))

fig, axs = plt.subplots(1,3, tight_layout=True)
mark_size=3
axs[0].plot(skews_good, means_good, 'bo', ms=mark_size, alpha=0.5)
axs[0].plot(skews_bad, means_bad, 'ro', ms=mark_size, alpha=0.5)

axs[1].plot(good_circs, means_good, 'bo', ms=mark_size, alpha=0.5)
axs[1].plot(bad_circs, means_bad, 'ro', ms=mark_size, alpha=0.5)

axs[2].plot(medians_good, means_good, 'bo', ms=mark_size, alpha=0.5)
axs[2].plot(medians_bad, means_bad, 'ro', ms=mark_size, alpha=0.5)
plt.show()

In [ ]:
fig, axs = plt.subplots()
mark_size=4

plt.plot(skews_good, st_devs_good, 'bo', ms=mark_size, alpha=0.5)
plt.plot(skews_bad, st_devs_bad, 'ro', ms=mark_size, alpha=0.5)
plt.show()

In [ ]:

ax = plt.figure().add_subplot(projection='3d')

# Plot a sin curve using the x and y axes.
x = np.linspace(0, 1, 100)
y = np.sin(x * 2 * np.pi) / 2 + 0.5
ax.scatter(kurts_good, good_circs, st_devs_good, zdir='z')
ax.scatter(kurts_bad, bad_circs, st_devs_bad, zdir='z')
# Customize the view angle so it's easier to see that the scatter points lie
# on the plane y=0
ax.view_init(elev=20., azim=-35, roll=0)

plt.show()

In [ ]:
model = lm.LogisticRegression(random_state=0)

In [ ]:
# Combine into a feature matrix and target vector y
features = np.column_stack((circs, circs_sharp, st_devs, kurts, skews, medians, means))
good = np.array(status_array_bool)

X_train, X_test, y_train, y_test = train_test_split(features, good, test_size=0.2, random_state=0)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


In [ ]:
# Combine into a feature matrix and target vector y
features = np.column_stack((circs, circs_sharp, st_devs, kurts, skews, medians, means))
good = np.array(status_array_bool)

X_train, X_test, y_train, y_test = train_test_split(features, good, test_size=0.2, random_state=0)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


In [ ]:
neus = [[4,6,9, 32,58,65],[2,3,14, 15,18,26]]
neu_good = traces[6]
neu_bad = traces[2]
rising = 1
falling = 1
rises = []
falls = []
rise_fall_ratios = []
mean_spikes = []
for neu in np.arange(len(traces)):
    rises = []
    falls = []
    neur = traces[neu]
    if neur[0]>neur[1]:
        rising = 0
        falling = 1
    else:
        rising=1
        falling=0
    spikes = []
    spk_thr = np.mean(neur)+np.std(neur)
    spike_max = 0
    for t in np.arange(1, len(neu_good)-2):
        if neur[t+1] < neur[t+2]:
            rising += 1
            if falling>0:
                if len(rises)>0 & (spike_max>spk_thr):
                    spikes.append(round(falling/rises[-1],2))
                falls.append(falling)
                falling=0
        else:
            falling += 1
            if rising>0:
                spike_max = neur[t+1]
                rises.append(rising)
                rising = 0
#    print(np.mean(spikes))
    mean_spikes.append(np.mean(spikes))




In [ ]:
fig, axs = plt.subplots(4,2, sharey=True, tight_layout=True)
pl=0
while pl <4:
    for n in np.arange(len(rise_fall_ratios)):
        if pl > 3:
            break
        if rise_fall_ratios[n]<0.217:
            print("hop")
            axs[pl,0].plot(traces[n])
            axs[pl,0].set_title(f"neuron {n+1}")
            pl+=1

pl=0
while pl <4:
    for n in np.arange(len(rise_fall_ratios)):
        if pl > 3:
            break
        if rise_fall_ratios[n]>0.24:
            print("ho")
            axs[pl,1].plot(traces[n])
            axs[pl,1].set_title(f"neuron {n+1}")
            pl+=1

plt.show()
